In [ ]:
file = open("KER.txt", "r")
content = file.read()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunked_docs = text_splitter.create_documents([content])
len(chunked_doc)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
'''from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})'''

In [ ]:
'''persist_directory = 'db'

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=instructor_embeddings)'''

In [ ]:
persist_directory = 'db'

vectordb = Chroma.from_documents(chunked_docs, 
                                 embedding_function=embeddings,
                                 persist_directory=persist_directory)
vectordb.persist()

In [ ]:
# test retreival

In [ ]:
question = "Explain the concept of heat equity"


docs = vectordb.similarity_search_with_score(question,k=3)

for result in docs:
    print("\n")
    print(result[1])
    print(result[0].page_content)

In [ ]:
question = "Who are some vulnerable populations in Arizona "


docs = vectordb.similarity_search_with_score(question,k=3)

for result in docs:
    print("\n")
    print(result[1])
    print(result[0].page_content)

In [ ]:
question = "What is the need for a Chief Heat Officer "


docs = vectordb.similarity_search_with_score(question,k=3)

for result in docs:
    print("\n")
    print(result[1])
    print(result[0].page_content)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """
[INST] <>
Use the following context to Answer the question at the end. Do not use any other information. If you can't find the relevant information in the context, just say you don't have enough information to answer the question. Don't try to make up an answer.

<>

{context}

Question: {question} [/INST]
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    # retriever=db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'k': 5, 'score_threshold': 0.8})
    # Similarity Search is the default way to retrieve documents relevant to a query, but we can use MMR by setting search_type = "mmr"
    # k defines how many documents are returned; defaults to 4.
    # score_threshold allows to set a minimum relevance for documents returned by the retriever, if we are using the "similarity_score_threshold" search type.
    # return_source_documents=True, # Optional parameter, returns the source documents used to answer the question
    retriever=vectordb.as_retriever(), # (search_kwargs={'k': 5, 'score_threshold': 0.8}),
    chain_type_kwargs={"prompt": prompt},
)
query = "Explain the concept of heat equity?"
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=100))

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
chunked_docs = text_splitter.create_documents([content])
len(chunked_doc)

In [ ]:
persist_directory = "db2/"
big_chunks_retriever = Chroma.from_documents(chunked_docs, 
                                 embedding_function=embeddings,
                                 persist_directory=persist_directory)
big_chunks_retriever.persist()

In [ ]:
question = "Explain the concept of heat equity"


docs = vectordb.similarity_search_with_score(question,k=3)

for result in docs:
    print("\n")
    print(result[1])
    print(result[0].page_content)

In [ ]:
question = "Who are some vulnerable populations in Arizona "


docs = vectordb.similarity_search_with_score(question,k=3)

for result in docs:
    print("\n")
    print(result[1])
    print(result[0].page_content)

In [ ]:
question = "What is the need for a Chief Heat Officer "


docs = vectordb.similarity_search_with_score(question,k=3)

for result in docs:
    print("\n")
    print(result[1])
    print(result[0].page_content)

In [ ]:
refine = RetrievalQA.from_chain_type(llm=OpenAI(),
                                 chain_type="map_reduce",
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs,
                                 retriever=big_chunks_retriever,
                                 verbose=True)

In [ ]:
query = "When was the solar system formed?"
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=100))

In [1]:
# new package for PDFs 
# Untested 
# Needs Llama Index

In [ ]:
from llmsherpa.readers import LayoutPDFReader

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_url = "knowledgeexchangeforresilience.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)

In [ ]:
from llama_index.readers.schema.base import Document
from llama_index import VectorStoreIndex

index = VectorStoreIndex([])
for chunk in doc.chunks():
    index.insert(Document(text=chunk.to_context_text(), extra_info={}))
query_engine = index.as_query_engine()

# Let's run one query
response = query_engine.query("What is the need for a Chief Heat Officer ")
print(response)